In [1]:
!pip install --upgrade streamlit pyngrok -q
!pip install pinecone -q
!pip install --upgrade pydantic -q
!pip install -U bitsandbytes -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.9/421.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/

In [10]:
!ngrok config add-authtoken 2wOlwfSCyrR717tnSJ9QLwrzFPx_6FEiQuNMhmGtQRFnkyie1

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
%%writefile app.py

import streamlit as st
from bs4 import BeautifulSoup
import requests
import time
import pinecone
import os
from pinecone import Pinecone, ServerlessSpec
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, AutoModel
from langchain.text_splitter import SpacyTextSplitter
import torch
import numpy as np


st.title('Review QnA')

def scrape_reviews(company_name, sources=['bestcompany', 'trustpilot'], num_pages=5, delay=2):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    all_reviews = []

    source_configs = {
        'bestcompany': {
            'url_template': f'https://bestcompany.com/health-insurance/{company_name}?page=',
            'selector': ('div', {'class_': 'whitespace-pre-line break-words'}),
        },
        'trustpilot': {
            'url_template': f'https://www.trustpilot.com/review/www.{company_name}.com?page=',
            'selector': ('p', {'attrs': {'data-service-review-text-typography': 'true'}}),
        }
    }

    for source in sources:
        if source not in source_configs:
            print(f"Source '{source}' is not supported.")
            continue

        config = source_configs[source]
        url_template = config['url_template']
        selector_type, selector_attrs = config['selector']

        def scrape_page(url):
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')
                if selector_type == 'p' and 'attrs' in selector_attrs:
                    reviews = soup.find_all(selector_type, **selector_attrs['attrs'])
                else:
                    reviews = soup.find_all(selector_type, **selector_attrs)
                return [review.get_text() for review in reviews]
            else:
                print(f"Error scraping {url}. Status code: {response.status_code}")
                return []

        url = url_template + str(1)
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            st.write(f"Company '{company_name}' not found on {source}.")
            continue

        source_reviews = []
        for page in range(1, num_pages + 1):
            url = url_template + str(page)
            st.write(f"Scraping {source} page {page}...")
            reviews = scrape_page(url)
            if not reviews:
                break
            source_reviews.extend(reviews)
            time.sleep(delay)

        all_reviews.extend(source_reviews)
        print(f"Added {len(source_reviews)} reviews from {source}")

    return all_reviews

@st.cache_resource
def models():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    embedding_tokenizer = AutoTokenizer.from_pretrained('jinaai/jina-embeddings-v3')
    embedding_model = AutoModel.from_pretrained('jinaai/jina-embeddings-v3', trust_remote_code=True).to(device)

    model_path = "microsoft/Phi-4-mini-instruct"
    chat_model = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map="cuda",
        load_in_8bit=True,
    )

    chat_tokenizer = AutoTokenizer.from_pretrained(model_path)
    st.write("Models loaded")
    return embedding_model, embedding_tokenizer, chat_model, chat_tokenizer

def vector_db():
    pine_key = "pcsk_6CjF3M_TVzGxF6Fnzq9mD1eFk33hRRRSrjysqAP4ndVRDNGxNfFkeKw3tq9TtwLz2BhuQT"
    pc = Pinecone(api_key=pine_key)
    index_name = 'store-embeddings'

    if index_name not in pc.list_indexes().names():
        pc.create_index(
            name=index_name,
            dimension=1024,
            metric="cosine",
            spec=ServerlessSpec(cloud="aws", region="us-east-1"))

    index = pc.Index(index_name)
    time.sleep(1)
    return index

def generate_embeddings(texts, embedding_model, embedding_tokenizer):
    embedding_model.eval()
    with torch.no_grad():
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        inputs = embedding_tokenizer(texts, padding=True, truncation=True, return_tensors='pt').to(device)
        outputs = embedding_model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1).cpu().float().numpy()
    return embeddings

def upserting(all_reviews, index):
    batch_size = 32
    text_splitter = SpacyTextSplitter(chunk_size=500)

    for review_id, review in enumerate(all_reviews):
        chunks = text_splitter.split_text(review)
        chunk_embeddings = []
        chunk_ids = []
        chunk_metadata = []

        for chunk_index, chunk in enumerate(chunks):
            chunk_embeddings.append(chunk)
            chunk_ids.append(f"{review_id}_{chunk_index}")
            chunk_metadata.append({"review_id": review_id, "chunk_index": chunk_index, "text": chunk})

            if len(chunk_embeddings) == batch_size:
                embeddings = generate_embeddings(chunk_embeddings)
                vectors = list(zip(chunk_ids, embeddings, chunk_metadata))
                index.upsert(vectors)
                chunk_embeddings, chunk_ids, chunk_metadata = [], [], []

        if chunk_embeddings:
            embeddings = generate_embeddings(chunk_embeddings)
            vectors = list(zip(chunk_ids, embeddings, chunk_metadata))
            index.upsert(vectors)

    st.write("Upserting Completed")

def retrieve_context(query, index, embedding_model):
    query_embedding = embedding_model.encode(query).tolist()
    results = index.query(vector = query_embedding, top_k=5, include_metadata=True)
    retrieved_text = " ".join([result['metadata']['text'] for result in results['matches']])
    return retrieved_text

def generate_response(query, retrieved_context, chat_model, chat_tokenizer):
    prompt = f"""### Context:
            {retrieved_context}

    ### Instructions:
    You are a helpful assistant answering questions based solely on the provided context.
    Respond in the third person without referencing individuals in the context.
    If the information needed to answer the question is not in the context, respond with "I cannot comment on that".
    Do not use any knowledge beyond what is in the context.

    ### Question:
    {query}

    ### Answer:
    """
    inputs = chat_tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = chat_model.generate(**inputs, max_new_tokens=150, temperature = 0.1, do_sample=True)
    return chat_tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)


# UI input
company = st.text_input("Enter Company Name")

if company:
    reviews = scrape_reviews(company, sources=['bestcompany', 'trustpilot'], num_pages=1)
    st.write(f"Total reviews scraped: {len(reviews)}")

    if reviews:
        embedding_model, embedding_tokenizer, chat_model, chat_tokenizer = models()
        print("Models Loaded")
        index = vector_db()
        # upserting(reviews, index)

        query = st.text_input("Enter a query")
        if query:
             context = retrieve_context(query, index, embedding_model)
            #st.write(context)
            #st.write("Retrieval temporarily disabled.")
             response = generate_response(query, context, chat_model, chat_tokenizer)
             st.write("Respomse")
             st.write(response)
        else:
            st.write("Please enter a query")
else:
    st.write("Please enter company name")


Writing app.py


In [7]:
ngrok.kill()

In [11]:
from pyngrok import ngrok

ngrok.kill()

!streamlit run app.py &> streamlit_log.txt &

public_url = ngrok.connect(8501, bind_tls = True)
print("Streamlit app is live at:", public_url)

Streamlit app is live at: NgrokTunnel: "https://7245-34-73-73-169.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
%%writefile app.py

import streamlit as st
from bs4 import BeautifulSoup
import requests
import time
import pinecone
from pinecone import Pinecone, ServerlessSpec
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel
from langchain.text_splitter import SpacyTextSplitter
import torch


# Function to scrape reviews
def scrape_reviews(company_name, sources=['bestcompany', 'trustpilot'], num_pages=5, delay=2):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    all_reviews = []

    source_configs = {
        'bestcompany': {
            'url_template': f'https://bestcompany.com/health-insurance/{company_name}?page=',
            'selector': ('div', {'class_': 'whitespace-pre-line break-words'}),
        },
        'trustpilot': {
            'url_template': f'https://www.trustpilot.com/review/www.{company_name}.com?page=',
            'selector': ('p', {'attrs': {'data-service-review-text-typography': 'true'}}),
        }
    }

    for source in sources:
        if source not in source_configs:
            print(f"Source '{source}' is not supported.")
            continue

        config = source_configs[source]
        url_template = config['url_template']
        selector_type, selector_attrs = config['selector']

        def scrape_page(url):
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')
                if selector_type == 'p' and 'attrs' in selector_attrs:
                    reviews = soup.find_all(selector_type, **selector_attrs['attrs'])
                else:
                    reviews = soup.find_all(selector_type, **selector_attrs)
                return [review.get_text() for review in reviews]
            else:
                print(f"Error scraping {url}. Status code: {response.status_code}")
                return []

        url = url_template + str(1)
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            st.write(f"Company '{company_name}' not found on {source}.")
            continue

        source_reviews = []
        for page in range(1, num_pages + 1):
            url = url_template + str(page)
            st.write(f"Scraping {source} page {page}...")
            reviews = scrape_page(url)
            if not reviews:
                break
            source_reviews.extend(reviews)
            time.sleep(delay)

        all_reviews.extend(source_reviews)
        print(f"Added {len(source_reviews)} reviews from {source}")

    return all_reviews


# Load models function
@st.cache_resource
def models():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    embedding_tokenizer = AutoTokenizer.from_pretrained('jinaai/jina-embeddings-v3')
    embedding_model = AutoModel.from_pretrained('jinaai/jina-embeddings-v3', trust_remote_code=True).to(device)

    model_path = "microsoft/Phi-4-mini-instruct"
    chat_model = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map="cuda",
        load_in_8bit=True,
    )

    chat_tokenizer = AutoTokenizer.from_pretrained(model_path)
    st.write("Models loaded")
    return embedding_model, embedding_tokenizer, chat_model, chat_tokenizer


# Pinecone setup and index handling
def vector_db():
    pine_key = "pcsk_7TbUb2_TigxRVhcCmVZDxxrNoMZEwePrwMjEUTobVHGsKzUdzJtqMZETWwh6w4UoB4gjbk"
    pc = Pinecone(api_key=pine_key)
    index_name = 'store-embeddings'

    # Check if the index exists
    if index_name not in pc.list_indexes().names():
        # If index doesn't exist, create it
        pc.create_index(
            name=index_name,
            dimension=1024,
            metric="cosine",
            spec=ServerlessSpec(cloud="aws", region="us-east-1")
        )

    index = pc.Index(index_name)
    time.sleep(1)
    return index


def check_index_status(index):
    try:
        index.describe()
        return True  # Index exists
    except Exception as e:
        return False  # Index does not exist


def delete_index():
    pine_key = "pcsk_7TbUb2_TigxRVhcCmVZDxxrNoMZEwePrwMjEUTobVHGsKzUdzJtqMZETWwh6w4UoB4gjbk"
    pc = Pinecone(api_key=pine_key)
    index_name = 'store-embeddings'
    try:
        pc.delete_index(index_name)
        st.write(f"Index '{index_name}' deleted successfully.")
    except Exception as e:
        st.write(f"Error deleting index: {e}")


# Function to generate embeddings
def generate_embeddings(texts, embedding_model, embedding_tokenizer):
    embedding_model.eval()
    with torch.no_grad():
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        inputs = embedding_tokenizer(texts, padding=True, truncation=True, return_tensors='pt').to(device)
        outputs = embedding_model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1).cpu().float().numpy()
    return embeddings


# Function to upsert reviews into Pinecone index
def upserting(all_reviews, index, embedding_model, embedding_tokenizer):
    batch_size = 32
    text_splitter = SpacyTextSplitter(chunk_size=500)

    for review_id, review in enumerate(all_reviews):
        chunks = text_splitter.split_text(review)
        chunk_embeddings = []
        chunk_ids = []
        chunk_metadata = []

        for chunk_index, chunk in enumerate(chunks):
            chunk_embeddings.append(chunk)
            chunk_ids.append(f"{review_id}_{chunk_index}")
            chunk_metadata.append({"review_id": review_id, "chunk_index": chunk_index, "text": chunk})

            if len(chunk_embeddings) == batch_size:
                embeddings = generate_embeddings(chunk_embeddings, embedding_model, embedding_tokenizer)
                vectors = list(zip(chunk_ids, embeddings, chunk_metadata))
                index.upsert(vectors)
                chunk_embeddings, chunk_ids, chunk_metadata = [], [], []

        if chunk_embeddings:
            embeddings = generate_embeddings(chunk_embeddings, embedding_model, embedding_tokenizer)
            vectors = list(zip(chunk_ids, embeddings, chunk_metadata))
            index.upsert(vectors)

    st.write("Upserting Completed")


# Retrieve context for a query
def retrieve_context(query, index, embedding_model):
    query_embedding = embedding_model.encode(query).tolist()
    results = index.query(vector=query_embedding, top_k=10, include_metadata=True)
    retrieved_text = " ".join([result['metadata']['text'] for result in results['matches']])
    return retrieved_text


# Generate response using the Phi model
def generate_response(query, retrieved_context, chat_model, chat_tokenizer):
    prompt = f"""### Context:
            {retrieved_context}

    ### Instructions:
    You are a helpful assistant answering questions based solely on the provided context.
    Respond in the third person without referencing individuals in the context.
    If the information needed to answer the question is not in the context, respond with "I cannot comment on that".
    Do not use any knowledge beyond what is in the context.

    ### Question:
    {query}

    ### Answer:
    """
    inputs = chat_tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = chat_model.generate(**inputs, max_new_tokens=150, temperature=0.1, do_sample=True)
    return chat_tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)


# UI input
company = st.text_input("Enter Company Name")

# Initialize Pinecone index
index = vector_db()

# Check and display index status
if not check_index_status(index):
    st.write("Database not initiated.")
else:
    st.write(f"Current status: Index is available.")

# Session flag to check if the delete button was clicked
if "delete_clicked" not in st.session_state:
    st.session_state.delete_clicked = False

# Main App Logic (only runs if delete wasn't just triggered)
if company and not st.session_state.delete_clicked:
    status_placeholder = st.empty()  # Placeholder for scraping status
    reviews = []

    with st.spinner("Scraping reviews..."):
        reviews = scrape_reviews(company, sources=['bestcompany', 'trustpilot'], num_pages=1)

    status_placeholder.empty()
    st.write(f"Total reviews scraped: {len(reviews)}")

    if reviews:
        embedding_model, embedding_tokenizer, chat_model, chat_tokenizer = models()
        print("Models Loaded")

        if not check_index_status(index):
            st.write("Index not found. Creating and initializing the database...")
            index = vector_db()
            upserting(reviews, index, embedding_model, embedding_tokenizer)
            st.write("Index created and reviews upserted.")

        query = st.text_input("Enter a query")
        if query:
            context = retrieve_context(query, index, embedding_model)
            response = generate_response(query, context, chat_model, chat_tokenizer)
            st.write("Response")
            st.write(response)

            # Bottom row with Delete Button on left
            col1, _ = st.columns([1, 8])
            with col1:
                if st.button("🗑️ Delete Database"):
                    delete_index()
                    st.session_state.delete_clicked = True
                    st.rerun()
        else:
            st.write("Please enter a query")
else:
    st.write("Please enter company name")

# Handle delete_clicked rerun behavior cleanly
if st.session_state.delete_clicked:
    st.write("Index deleted. You can now reinitialize with new data.")
    # Optional: Add a "Reset" button
    if st.button("🔁 Reset App"):
        st.session_state.delete_clicked = False
        st.rerun()


Overwriting app.py
